In [ ]:
import pandas as pd

In [ ]:
appeal = pd.read_csv('/content/drive/MyDrive/Hackathon/appeal.csv')
appeal_doc = pd.read_csv('/content/drive/MyDrive/Hackathon/appeal_document.csv')

appeal_doc = appeal_doc[appeal_doc.name.str.lower().str.contains('final')]
appeal = appeal[appeal['dtype.name'] == 'Flood']
appeal_doc = appeal_doc[appeal_doc.appeal.isin(appeal.code.unique().tolist())]

In [ ]:
# Initialize empty lists to store extracted information
total_expenditure = []
extra_list = []
total_contributions = []

#We have identified some words manually to be in data with which 'Total' word should not come, using them below
should_be_present = [
    "Contribution",
    "Programme",
    "Consultants",
    "Indirect",
    "Land",
    "Transport",
    "Operational",
    "Personnel",
    "Programme",
    "Relief",
    "Services",
    "Supplies",
    "Workshops"
]

# Iterate through the unique document URLs
for appeal_url in appeal_doc.document_url.unique().tolist():
    # Download the PDF document using the 'download_file' function
    appeal = download_file(appeal_url)

    # Open the PDF using pdfplumber
    with pdfplumber.open(appeal) as pdf:
        # Extract pages from the PDF
        pages = pdf.pages
        list_text = []

        # Iterate through the pages and extract text
        for page in pages:
            text = page.extract_text()
            list_text.append(text)

    # Initialize empty lists to store specific information
    new_list = []
    item_list = []
    contribution_list = []
    # Iterate through the extracted text from the PDF
    for i in list_text:
        # Split the text into lines
        a = i.split('\n')

        # Check for items that should be present and exclude lines with 'Total'
        for j in a:
            if any(item in j for item in should_be_present):
                item_list.append(j)
            if 'Total Expenditure' in j or 'TOTAL EXPENDITURE' in j:
                new_list.append(j)
            if 'Total Contribution' in j or 'TOTAL CONTRIBUTION' in j:
              contribution_list.append(j)

    # Append the appeal URL and 'Total Expenditure' lines to the 'total_expenditure' list
    total_expenditure.append([appeal_url, new_list])

    # Append the appeal URL and lines with specific items to the 'extra_list' list
    extra_list.append([appeal_url, item_list])

    # Append the appeal URL and lines with specific items to the 'contribution_list' list
    total_contributions.append([appeal_url,contribution_list])


In [ ]:
#Creating dataframe for list which store 'Total Contribution' values

def remove_periods_and_quotes(text):
    if text:
        return text.replace('.', '').replace("'", '')
    else:
        return ''

df = pd.DataFrame(total_contributions, columns=["document_url", "Total Contributions"])
max_num_columns = max(len(text[0].split()) for text in df["Total Contributions"] if text) - 4

def process_text(text_list, desired_length):
    # Check if the text list is empty
    if not text_list:
        return [''] * desired_length

    # Define the prefix to remove
    prefix = 'Total Contributions & Transfers '
    prefix2 = 'Total Contributions and Other Income '
    # Remove the prefix from the single item in the list
    processed_text = text_list[0].replace(prefix, '')
    processed_text = processed_text.replace(prefix2, '')

    # Split the text by whitespace
    split_text = processed_text.split()

    # Check if the split_text has fewer elements than the desired length
    if len(split_text) < desired_length:
        # Add empty strings to match the desired length
        split_text.extend([''] * (desired_length - len(split_text)))

    return split_text

# Split the "Text" column and add the resulting values to different columns

for i in range(max_num_columns):
    df[f"Total Contributions {i+1}"] = df["Total Contributions"].apply(lambda x: process_text(x,max_num_columns)[i])



# Apply the remove_periods_and_quotes function to all columns with names starting with 'Total Expenditure '
for i in range(max_num_columns):
    column_name = f"Total Contributions {i+1}"
    df[column_name] = df[column_name].apply(remove_periods_and_quotes)

prefix = 'Total Contributions & Transfers '
prefix2 = 'Total Contributions and Other Income '

# Create a function to check the prefix within each list
def get_itemname(lst):
    for item in lst:
        if item.startswith(prefix):
            return prefix
        elif item.startswith(prefix2):
            return prefix2
    return None  # Return None if neither prefix is found

# Apply the function to each row in the DataFrame
df['Itemname'] = df['Total Contributions'].apply(get_itemname)

In [ ]:
#Creating dataframe for list which store 'Total Expenditure' values

df_expenditure = pd.DataFrame(total_expenditure, columns=["document_url", "Total Expenditure"])
max_num_columns = max(len(text[0].split()) for text in df["Total Expenditure"] if text) - 3

def process_text(text_list, desired_length):
    # Check if the text list is empty
    if not text_list:
        return [''] * desired_length

    # Define the prefix to remove
    prefix = 'TOTAL EXPENDITURE (D)'

    # Remove the prefix from the single item in the list
    processed_text = text_list[0].replace(prefix, '')

    # Split the text by whitespace
    split_text = processed_text.split()

    # Check if the split_text has fewer elements than the desired length
    if len(split_text) < desired_length:
        # Add empty strings to match the desired length
        split_text.extend([''] * (desired_length - len(split_text)))

    return split_text

# Split the "Text" column and add the resulting values to different columns

for i in range(max_num_columns):
    df_expenditure[f"Total Expenditure {i+1}"] = df_expenditure["Total Expenditure"].apply(lambda x: process_text(x,max_num_columns)[i])



# Apply the remove_periods_and_quotes function to all columns with names starting with 'Total Expenditure '
for i in range(max_num_columns):
    column_name = f"Total Expenditure {i+1}"
    df_expenditure[column_name] = df_expenditure[column_name].apply(remove_periods_and_quotes)

In [ ]:
#Creating dataframe for should_be_present list keywords

def ends_with_number(text):
    pattern = r'\d$'
    match = re.search(pattern, text)
    return match is not None


def split_text(text):
    # Split the text by whitespace
    words = text.split()

    # Initialize lists for alphabetic and numeric items
    alphabetic_items = []
    numeric_items = []

    # Iterate through the words
    for word in words:
        # Check if the word has alphabetic characters
        if re.search('[a-zA-Z]', word):
            # Join alphabetic words by ' ' and append to the alphabetic_items list
            alphabetic_items.append(word)
        else:
            # Check if the word contains only numbers, ',' or '.'
            if re.match(r"^[0-9,.'\"]+$", word):
                # Append numeric words to the numeric_items list
                numeric_items.append(word)

    # Join the alphabetic items by ' '
    alphabetic_text = ' '.join(alphabetic_items)

    # Return both the alphabetic and numeric items
    return alphabetic_text, numeric_items


def find_max_numeric_length(texts):
    max_length = -1
    max_length_text = None

    for text in texts:
        _, numeric_items = split_text(text)
        numeric_length = len(numeric_items)

        if numeric_length > max_length:
            max_length = numeric_length
            max_length_text = text

    return max_length

flat_list = []
for i in extra_list:
  output_list = i[1]
  output_list = [i for i in output_list if i.startswith(tuple(should_be_present))]
  output_list = list(filter(ends_with_number, output_list))
  for k in output_list:
    flat_list.append(k)

max_length = find_max_numeric_length(flat_list)
df_ = pd.DataFrame(columns=['document_url','text'] + [f'number_{i}' for i in range(max_length)])
# Process each item in extra_list
for url, output_list in extra_list:
    # Extract the URL
    output_list = [i for i in output_list if i.startswith(tuple(should_be_present))]
    output_list = list(filter(ends_with_number, output_list))
    for item in output_list:
      text, numbers = split_text(item)
      # Create a dictionary to represent a row

      row_data = {'document_url':url,'text': text}

      # Add the individual items from the 'numbers' list as columns
      for i in range(max_length):
          if i < len(numbers):
            row_data[f'number_{i}'] = numbers[i]
          else:
            row_data[f'number_{i}'] = None  # Fill missing values with None or any other suitable value

      # Append the row to the DataFrame
      df_ = df_.append(row_data, ignore_index=True)


def remove_periods_and_quotes(text):
    if text:
        return text.replace('.', '').replace("'", '')
    else:
        return ''

# Apply the remove_periods_and_quotes function to all columns with names starting with 'Total Expenditure '
for i in range(0,max_length):
    column_name = f"number_{i}"
    df_[column_name] = df_[column_name].apply(lambda x: remove_periods_and_quotes(str(x)))

In [ ]:
# Save all three dataframes

df.to_csv('Total_contributions_output.csv')
df_expenditure.to_csv('Total_expenditure_output.csv',index=False)
df_.to_csv('Not_Total_output.csv',index=False)

In [ ]:
# After some manual changes and pivoting the files to have data for one document_url in one row, we read files again
#Now these files, will be merged with appeal, appeal document and output of tabula flood file to provide full scale data for Flood at document level

df = pd.read_csv('Total_contributions.csv')
df_ = pd.read_csv('Not_Total_output.csv')
df_expenditure = pd.read_csv('Total_expenditure.csv')
df_tabula = pd.read_csv('Tabula_Flood_output.csv')

data = df.merge(df_expenditure,on='document_url',how='left')
data = data.merge(df_,on='document_url',how='left')
data = data.merge(df_tabula,on='document_url',how='left')
data = data.merge(appeal_doc,on='document_url',how='left')
data = data.merge(appeal,left_on='appeal',right_on='code',how='right')

#Saving the final file
data.to_csv('Final_Flood_output_file.csv')